# Exhibition - enriching information

In [1]:
import pandas as pd
import duckdb

In [2]:
df_exhibition = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_done/Artlas Exhibition.csv', dtype ='str')
df_catalog = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibition Catalog.csv', dtype ='str')
df_exhibition_section = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibition Section.csv', dtype ='str')
df_has_exhibited_in = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibited.csv', dtype ='str')
df_personage = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/personage/personage_uuid/uuid_exhibitor.csv', low_memory=False, dtype ='str')
df_artwork = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibited Work.csv', low_memory=False, dtype ='str')

In [3]:
df_catalog.rename(columns={'id': 'id_catalog'}, inplace=True)
df_exhibition.rename(columns={'id': 'id_exhibition'}, inplace=True)
df_exhibition_section.rename(columns={'id': 'id_section'}, inplace=True)
df_exhibition_section.rename(columns={'id_address': 'id_address_exhibition'}, inplace=True)
df_has_exhibited_in.rename(columns={'id_exhibition_section': 'id_section', 'id': 'id_has_exhibited_in'}, inplace=True)
df_artwork.rename(columns={'id': 'id_work'}, inplace=True)

In [4]:
df_personage.head(3)

,id_personage,name,first_name,id_birth_address,id_death_address,id_birth_date,id_death_date,gender,membership,id_type,...,death_address,biography,instructor,address1,id_address1,address2,id_address2,wikidata_id,Full_name,uuid
0,37218,Manganaris,Yannis,14647,NaN,1713,NaN,M,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yannis Manganaris,7c1dac39-2918-3cf8-bb24-3b9b194edede
1,49029,Abrams,Lionel,NaN,NaN,3097,NaN,M,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lionel Abrams,9e34b27c-7a96-32ed-bdf6-f2529ea23675
2,49030,Battiss,Walter,NaN,NaN,1090,NaN,M,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walter Batiss,d859377e-8bc5-3a5b-b3fb-e84c2dca8fcb


In [5]:
merged_catalog = pd.merge(df_exhibition[['id_exhibition']], df_catalog[['id_catalog', 'id_exhibition']], left_on='id_exhibition', right_on='id_exhibition')

In [6]:
merged_catalog.head(3)

,id_exhibition,id_catalog
0,8155,3697
1,8156,3698
2,18595,3908


In [7]:
merged_section = pd.merge(merged_catalog[['id_catalog', 'id_exhibition']], df_exhibition_section[['id_section', 'id_catalog', 'id_address_exhibition']], left_on='id_catalog', right_on='id_catalog')

In [8]:
merged_section.head(3)

,id_catalog,id_exhibition,id_section,id_address_exhibition
0,3697,8155,4007,18157
1,3698,8156,4008,15347
2,2675,2451,1482,12038


In [9]:
merged_exhibited = pd.merge(merged_section[['id_catalog', 'id_exhibition', 'id_section', 'id_address_exhibition']], df_has_exhibited_in[['id_personage', 'id_section', 'id_has_exhibited_in']], left_on='id_section', right_on='id_section')

In [10]:
merged_exhibited.head(3)

,id_catalog,id_exhibition,id_section,id_address_exhibition,id_personage,id_has_exhibited_in
0,3697,8155,4007,18157,50822,43377
1,3697,8155,4007,18157,50823,43378
2,3697,8155,4007,18157,50824,43379


In [11]:
merged_uuid_person = pd.merge(merged_exhibited[['id_catalog', 'id_exhibition', 'id_section','id_personage', 'id_address_exhibition', 'id_has_exhibited_in']], df_personage[['id_personage', 'uuid']], left_on='id_personage', right_on='id_personage')

In [12]:
merged_uuid_person.head(5)

,id_catalog,id_exhibition,id_section,id_personage,id_address_exhibition,id_has_exhibited_in,uuid
0,3697,8155,4007,50822,18157,43377,51e8e51d-7f79-34bf-a06a-a200b29773d3
1,3697,8155,4007,50823,18157,43378,7abca4df-0367-369c-87d9-7eb294055855
2,3697,8155,4007,50824,18157,43379,bbcf201d-01ad-328d-8e9a-9cd0fea515de
3,3697,8155,4007,50825,18157,43380,47379550-f14e-3982-830f-5b4f1f8b8537
4,3697,8155,4007,50826,18157,43381,c8098e5d-06cb-3fcc-b99f-1f9ecc30939c


In [13]:
merged_work = pd.merge(merged_uuid_person[['id_catalog', 'id_exhibition', 'id_section','id_personage', 'id_address_exhibition', 'id_has_exhibited_in', 'uuid']], df_artwork[['id_work', 'id_has_exhibited_in']], left_on='id_has_exhibited_in', right_on='id_has_exhibited_in')

In [14]:
merged_work.drop('id_has_exhibited_in', axis=1, inplace=True)

In [15]:
merged_work.head(3)

,id_catalog,id_exhibition,id_section,id_personage,id_address_exhibition,uuid,id_work
0,3697,8155,4007,50822,18157,51e8e51d-7f79-34bf-a06a-a200b29773d3,102296
1,3697,8155,4007,50822,18157,51e8e51d-7f79-34bf-a06a-a200b29773d3,102297
2,3697,8155,4007,50822,18157,51e8e51d-7f79-34bf-a06a-a200b29773d3,102298


In [16]:
merged_addr = pd.merge(merged_work[['id_catalog', 'id_exhibition', 'id_section', 'id_address_exhibition', 'id_personage', 'uuid', 'id_work']], df_personage[['id_personage', 'id_address1']], left_on='id_personage', right_on='id_personage')

In [17]:
merged_addr

,id_catalog,id_exhibition,id_section,id_address_exhibition,id_personage,uuid,id_work,id_address1
0,3697,8155,4007,18157,50822,51e8e51d-7f79-34bf-a06a-a200b29773d3,102296,NaN
1,3697,8155,4007,18157,50822,51e8e51d-7f79-34bf-a06a-a200b29773d3,102297,NaN
2,3697,8155,4007,18157,50822,51e8e51d-7f79-34bf-a06a-a200b29773d3,102298,NaN
3,3697,8155,4007,18157,50823,7abca4df-0367-369c-87d9-7eb294055855,102299,NaN
4,3697,8155,4007,18157,50824,bbcf201d-01ad-328d-8e9a-9cd0fea515de,102300,NaN
...,...,...,...,...,...,...,...,...
164455,4125,18773,4454,18555,92718,88b9171c-73f6-35da-8daa-7f05c7ed0572,257585,18336
164456,4125,18773,4454,18555,92719,552fbdc5-de0c-3399-964a-da8bc1646573,257586,15347
164457,4125,18773,4454,18555,92719,552fbdc5-de0c-3399-964a-da8bc1646573,257587,15347
164458,4125,18773,4454,18555,92720,8f351dcf-7b52-3953-a508-aa31f011a823,257588,NaN


# Saving the final results

In [18]:
columns = merged_addr.columns

# Iterate through the columns
for col in columns:
    # Check if the column name contains a space
    if ' ' in col:
        # Replace spaces with underscores
        merged_work.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [19]:
merged_addr.to_xml('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibition_enriched.xml', root_name="data", pretty_print=True, index=False)

# Data Analysis

Checking data consistency

## How many sections and catalogues per exhibition

In [38]:
final_merged_with_exhibition = pd.merge(merged_work, df_exhibition, on='id_exhibition', how='left')

In [39]:
duckdb.sql("CREATE TABLE exhibition AS SELECT * FROM final_merged_with_exhibition")

In [40]:
result = duckdb.execute("SELECT * FROM exhibition").fetchdf()
result.head(5)

,id_catalog,id_exhibition,id_section,id_personage,id_address_exhibition,id_work,title,traveling,source,id_user,timestamp,begin_year,end_year,id_city,id_exhibition_group,former_country,former_city,public
0,3697,8155,4007,50822,18157,102296,Brooklyn Loan Exhibit,f,archive.org,2,2018-12-29 10:40:27.262038,1911,1911,6812,1,NaN,NaN,f
1,3697,8155,4007,50822,18157,102297,Brooklyn Loan Exhibit,f,archive.org,2,2018-12-29 10:40:27.262038,1911,1911,6812,1,NaN,NaN,f
2,3697,8155,4007,50822,18157,102298,Brooklyn Loan Exhibit,f,archive.org,2,2018-12-29 10:40:27.262038,1911,1911,6812,1,NaN,NaN,f
3,3697,8155,4007,50823,18157,102299,Brooklyn Loan Exhibit,f,archive.org,2,2018-12-29 10:40:27.262038,1911,1911,6812,1,NaN,NaN,f
4,3697,8155,4007,50824,18157,102300,Brooklyn Loan Exhibit,f,archive.org,2,2018-12-29 10:40:27.262038,1911,1911,6812,1,NaN,NaN,f


In [45]:
query = """
SELECT id_exhibition, traveling, COUNT(DISTINCT id_section) as section_count, COUNT(DISTINCT id_address_exhibition) as id_address_exhibition_count , COUNT(DISTINCT id_catalog) as catalog_count_count
FROM exhibition
GROUP BY id_exhibition, traveling
HAVING COUNT(DISTINCT id_section) > 1;
"""

In [46]:
result = duckdb.query(query).fetchdf()
print(result)

   id_exhibition traveling  section_count  id_address_exhibition_count  \
0           5582         f             54                           54   
1           4322       NaN              2                            1   
2           8149         f              5                            1   
3          18437         f              2                            1   
4           5814         f              4                            1   
5           4157         f              2                            1   
6           4181         f              2                            1   
7           5781         f              2                            1   
8           5810         f              3                            1   
9           4079         f             40                            3   
10          3993         f              4                            1   
11         18424         t              4                            1   
12         18433         f            

### Multiple catalogues' id example

In [ ]:
catalogues = duckdb.execute("SELECT DISTINCT id_section, id_catalog FROM exhibition WHERE id_exhibition = 18492;").fetchdf()

In [ ]:
print(catalogues)

### Multiple section example

In [43]:
section = duckdb.execute("SELECT DISTINCT id_section, id_catalog FROM exhibition WHERE id_exhibition = 4283;").fetchdf()

In [44]:
print(section)

   id_section id_catalog
0        2784        291
1        2785        291
2        2786        291
3        2787        291
4        2788        291
5        2789        291
6        2790        291
7        3188        291
8        3189        291
9        3190        291
10       3191        291
11       3192        291
